In [1]:
from dotenv import load_dotenv
import warnings

from environment.game import Game
from environment.game_state import GameState
from environment.generators.random_generator import RandomCrimeSceneMapGenerator
from llm.llama import Llama
from llm.story_generator import StoryGenerator

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")
warnings.filterwarnings("ignore", category=FutureWarning)

/home/jovyan/my-conda-envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Create a room layout and a theme

In [2]:
number_of_rooms = 3
theme = "Smallville, Clark Kent, 2010"

crime_scene_map = RandomCrimeSceneMapGenerator().generate(number_of_rooms, 42)

2. Init Llama2

In [3]:
from langchain.llms import HuggingFacePipeline

llm = Llama()
llm_pipeline = HuggingFacePipeline(pipeline=llm.pipeline)

Loading checkpoint shards: 100%|██████████| 3/3 [00:29<00:00,  9.98s/it]


Model loaded on cuda:0


In [4]:
import json
import re
from time import time

from llm.chains.victim_chain import VictimChain

themes = [
    ['Prohibition Era', 'Speakeasy', 'Bootlegger'],
    ['Victorian London', 'Jack the Ripper', 'Scotland Yard'],
    ['1920s Chicago', 'Al Capone', 'Organized Crime'],
    ['Ancient Rome', 'Emperor Nero', 'Assassination Plot'],
    ['Futuristic Megacity', 'Cybernetic Hacker', 'Corporate Espionage'],
    ['Medieval Castle', 'Royal Poisoning', 'Intrigue'],
    ['Wild West', 'Outlaw Gang', 'Bank Heist'],
    ['Small Coastal Town', 'Missing Fisherman', 'Mysterious Disappearances'],
    ['Art World Heist', 'Stolen Masterpiece', 'Undercover Detective'],
    ['1940s Hollywood', 'Film Noir Detective', 'Murder Mystery'],
]

chain = VictimChain(llm_pipeline)

results = []
for theme in themes:
    start_time = time()
    victim = chain.create(theme)
    end_time = time()
    
    victim = re.sub('\n\s*', '', victim)
    
    res = {
        "elapsed_time": end_time - start_time,
        "theme": theme,
        "llm_output": victim
    }
    
    try:
        parsed = json.loads(victim)
        res.update({ "json_parsed": parsed })
    except Exception as e:
        res.update({ "error": repr(e) })
        
    results.append(res)

In [5]:
results

[{'elapsed_time': 9.760753393173218,
  'theme': ['Prohibition Era', 'Speakeasy', 'Bootlegger'],
  'llm_output': '{"name": "Vinnie "The Knife" LaRosa","age": 35,"occupation": "Notorious bootlegger and owner of The Blind Pig speakeasy","murder_weapon": "Silenced.45 caliber revolver","death_description": "Found slumped over his office desk with a look of shock on his face, a bullet hole in the center of his forehead, and a note reading \'You\'ve been pinched, Vinnie.\'"}',
  'error': 'JSONDecodeError("Expecting \',\' delimiter: line 1 column 19 (char 18)")'},
 {'elapsed_time': 6.899394273757935,
  'theme': ['Victorian London', 'Jack the Ripper', 'Scotland Yard'],
  'llm_output': '{"name": "Mary Ann Nichols","age": 45," occupation": "Prostitute","murder_weapon": "sharp knife","death_description": "Found lying on her back in a pool of blood, throat slit from ear to ear, body mutilated beyond recognition, in an alleyway off Whitechapel Road."}',
  'json_parsed': {'name': 'Mary Ann Nichols',


3. Create a story

In [4]:
story = StoryGenerator(crime_scene_map.rooms, llm).create_new_story(number_of_suspects=number_of_rooms - 1, theme=theme, dummy=False)

story

{'name': 'Archibald Ptolemy', 'age': 35, 'occupation': 'Head Librarian', 'murder_weapon': 'Ancient scroll with poisoned ink', 'death_description': 'Found dead in his office surrounded by stacks of books, face contorted in a mixture of fear and surprise, as if he had been reading a particularly gruesome text when struck down.'}

            killer:
              name: "Gaius"
              age: 40
              occupation: "Crazy Librarian"
              alibi: |
                Gaius, known as the 'Crazy Librarian,' claims he was in his secret chamber, delving into forbidden texts.
                No one can vouch for his whereabouts.
              motive: |
                Gaius had become increasingly obsessed with ancient and forbidden knowledge.
                He believed that by eliminating anyone who questioned him, he could protect the library's secrets.
              
{'name': 'Gaius', 'age': 40, 'occupation': 'Crazy Librarian', 'motive': "Gaius had become increasingly obsesse

{'theme': 'Library of Alexandria, 340 BC, crazy librarian',
 'killer': {'name': 'Gaius',
  'age': 40,
  'occupation': 'Crazy Librarian',
  'motive': "Gaius had become increasingly obsessed with ancient and forbidden knowledge.\nHe believed that by eliminating anyone who questioned him, he could protect the library's secrets.\n",
  'alibi': "Gaius, known as the 'Crazy Librarian,' claims he was in his secret chamber, delving into forbidden texts.\nNo one can vouch for his whereabouts.\n"},
 'victim': {'name': 'Archibald Ptolemy',
  'age': 35,
  'occupation': 'Head Librarian',
  'murder_weapon': 'Ancient scroll with poisoned ink',
  'death_description': 'Found dead in his office surrounded by stacks of books, face contorted in a mixture of fear and surprise, as if he had been reading a particularly gruesome text when struck down.'},
 'suspects': [{'name': 'Eutychius',
   'age': 28,
   'occupation': 'Junior Clerk',
   'motive': 'Eutychius held a grudge against Archibald due to constant cri

4. Create an initial game state

In [5]:
initial_game_state = GameState(crime_scene_map, story)

AttributeError: 'dict' object has no attribute 'victim'

5. Create a game

In [ ]:
game = Game(initial_game_state)